In [1]:
def read_file(_path, delim='\t'):
    data = []
    if delim == 'tab':
        delim = '\t'
    elif delim == 'space':
        delim = ' '
    with open(_path, 'r') as f:
        for line in f:
            line = line.strip().split(delim)
            line = [float(i) for i in line]
            data.append(line)
    return np.asarray(data)

In [2]:
def poly_fit(traj, traj_len, 
             threshold
             ):
    """
    Input:
    - traj: Numpy array of shape (2, traj_len)
    - traj_len: Len of trajectory
    - threshold: Minimum error to be considered for non linear traj
    Output:
    - int: 1 -> Non Linear 0-> Linear
    """
    t = np.linspace(0, traj_len - 1, traj_len) # np.linspace(시작점, 끝점, 구간 내 숫자 개수)
    res_x = np.polyfit(t, traj[0, -traj_len:], 2, full=True)[1]
    res_y = np.polyfit(t, traj[1, -traj_len:], 2, full=True)[1]
    if res_x + res_y >= threshold: # error
       return 1.0
    else:
       return 0.0
    return 0.0

In [3]:
import logging
import os
import math

import numpy as np


import torch
from torch.utils.data import Dataset
path = 'C:\\Users\\NGN\\dev\\Traffino\\TRAFFINO\\traffino\\datasets\\waterloo\\train\\0769_prep.txt'
path2 = 'C:\\Users\\NGN\\dev\\Traffino\\TRAFFINO\\traffino\\datasets\\waterloo\\train\\0769_prep2.txt'
path3 = 'C:\\Users\\NGN\\dev\\Traffino\\TRAFFINO\\traffino\\datasets\\waterloo\\train\\0769_prep3.txt'
delim = '\t'
data = read_file(path, delim)
data2 = read_file(path2, delim)
data3 = read_file(path3, delim)
print(len(data))

print(len(data))
print(type(data))

16387
16387
<class 'numpy.ndarray'>


In [4]:
frames = np.unique(data[:, 0]).tolist()
# print(frames[0])
print(len(frames))

727


In [5]:
frame_data_all = []
frame_data = []
state_data = []
traffic_data = []
# data를 frame 단위로 변환
for frame in frames:
    frame_data_all.append(data[frame == data[:, 0], :4])  # all
    frame_data.append(data[frame == data[:, 0], :4]) # 전체 frame에(7994개 scene) 대해, 각 frame별 frame_data (agent 정보, x, y) --> 2차원 list
    state_data.append(data2[frame == data2[:, 0], :])
    traffic_data.append(data3[frame == data3[:, 0], ])

In [6]:
print(len(frame_data))
print(type(frame_data))
# print(frame_data)
# print(len(frame_data[0]))

727
<class 'list'>


In [7]:
print(len(state_data))
print(type(state_data))
# print(state_data)
# print(len(state_data[0]))

727
<class 'list'>


In [8]:
print(len(traffic_data))
print(type(traffic_data))
# print(traffic_data)
# print(len(state_data[0]))

727
<class 'list'>


In [13]:
# print(frame_data[0:20])
# print(len(frame_data[0:20]))

In [9]:
seq_len=20
skip=1

num_sequences = int(math.ceil((len(frames) - seq_len + 1) / skip)) # seqence 개수

In [10]:
obs_len = 8
pred_len = 12
threshold = 0.002
min_agent =1 
num_agents_in_seq = []
seq_list = []
seq_list2 = [] # state
seq_list3 = [] # traffic 

non_linear_agent = []
loss_mask_list = []
seq_list_rel = []
seq_list_rel2 = [] # state
seq_list_rel3 = [] # traffic 



for idx in range(0, num_sequences * skip + 1, skip):
    curr_seq_data = np.concatenate(                         # num_sequnce만큼 끊어서 curr seq_data 생성
        frame_data[idx:idx + seq_len], axis=0               # ex) seq 1: 1-20 frame, seq 2: 2-21 frame ... 
    )
    curr_seq_data2 = np.concatenate(                         # num_sequnce만큼 끊어서 curr seq_data 생성
        state_data[idx:idx + seq_len], axis=0               # ex) seq 1: 1-20 frame, seq 2: 2-21 frame ... 
    )
    curr_seq_data3 = np.concatenate(                         # num_sequnce만큼 끊어서 curr seq_data 생성
        traffic_data[idx:idx + seq_len], axis=0               # ex) seq 1: 1-20 frame, seq 2: 2-21 frame ... 
    )
    # print(idx) # 7976 (sequnece 개수)
    # print(curr_seq_data)
    # print(len(curr_seq_data)) # 현재 시퀀스에 있는 row (프레임 별 agent)의 합 --> 20개 프레임에 대한 agent (중복 포함) 합 --> 20*20= 400

    agents_in_curr_seq = np.unique(curr_seq_data[:, 1])       # 현재 seq에 있는 agents 목록, 모든 행의 1번째(agent 정보) 열 slicing

    # print(len(agents_in_curr_seq))
    # print(agents_in_curr_seq)
    curr_seq_rel = np.zeros((len(agents_in_curr_seq), 2,        # (현재 seq에 있는 agents 개수, 2, seq_len) 
                                seq_len))
    curr_seq_rel2 = np.zeros((len(agents_in_curr_seq), 4,        # (현재 seq에 있는 agents 개수, 4, seq_len) --> state 저장용
                            seq_len))
    curr_seq_rel3 = np.zeros((len(agents_in_curr_seq), 1,        # (현재 seq에 있는 agents 개수, 1, seq_len) --> traffic light 저장용
                        seq_len))
    
    # print(curr_seq_rel)
    curr_seq = np.zeros((len(agents_in_curr_seq), 2, seq_len))   # (현재 seq에 있는 agents 개수, 2, seq_len) 
    curr_seq2 = np.zeros((len(agents_in_curr_seq), 4, seq_len))   # (현재 seq에 있는 agents 개수, 4, seq_len) --> state 저장용
    curr_seq3 = np.zeros((len(agents_in_curr_seq), 1, seq_len))   # (현재 seq에 있는 agents 개수, 3, seq_len) --> traffic light 저장용
    
    curr_loss_mask = np.zeros((len(agents_in_curr_seq),
                                seq_len))
    num_agents_considered = 0
    _non_linear_agent = []
    
    for _, agent_id in enumerate(agents_in_curr_seq):           # 현재 seq에 있는 agents 목록
        curr_agent_seq = curr_seq_data[ curr_seq_data[:, 1] == # 현재 seq에 있는 agent의 위치 seqeunce curr_agent_seq: (20, 4)
                                        agent_id, :]
        curr_agent_seq2 = curr_seq_data2[ curr_seq_data2[:, 1] == # 현재 seq에 있는 agent의 상태 seqeunce curr_agent_seq: (20, 4)
                                agent_id, :]
        curr_agent_seq3 = curr_seq_data3[ curr_seq_data3[:, 1] == # 현재 seq에 있는 agent의 traffic seqeunce curr_agent_seq: (20, 4)
                                agent_id, :]
        # print(curr_agent_seq)
        # print(agent_id)
        curr_agent_seq = np.around(curr_agent_seq, decimals=4)
        curr_agent_seq2 = np.around(curr_agent_seq2, decimals=4)
        curr_agent_seq3 = np.around(curr_agent_seq3, decimals=4)
                
        pad_front = frames.index(curr_agent_seq[0, 0]) - idx
        pad_end = frames.index(curr_agent_seq[-1, 0]) - idx + 1
        if pad_end - pad_front != seq_len:
            continue
        curr_agent_seq = np.transpose(curr_agent_seq[:, 2:])
        curr_agent_seq2 = np.transpose(curr_agent_seq2[:, 2:])
        curr_agent_seq3 = np.transpose(curr_agent_seq3[:, 2:])
        
        curr_agent_seq = curr_agent_seq
        # Make coordinates relative
        rel_curr_agent_seq = np.zeros(curr_agent_seq.shape)
        
        rel_curr_agent_seq[:, 1:] = \
            curr_agent_seq[:, 1:] - curr_agent_seq[:, :-1]
        _idx = num_agents_considered
        
        curr_seq[_idx, :, pad_front:pad_end] = curr_agent_seq
        curr_seq2[_idx, :, pad_front:pad_end] = curr_agent_seq2
        curr_seq3[_idx, :, pad_front:pad_end] = curr_agent_seq3
        
        curr_seq_rel[_idx, :, pad_front:pad_end] = rel_curr_agent_seq
        # Linear vs Non-Linear Trajectory
        _non_linear_agent.append(
            poly_fit(curr_agent_seq, pred_len 
                        ,threshold
                        )
                        )
        curr_loss_mask[_idx, pad_front:pad_end] = 1
        num_agents_considered += 1
        
    if num_agents_considered > min_agent:
        non_linear_agent += _non_linear_agent
        num_agents_in_seq.append(num_agents_considered)
        loss_mask_list.append(curr_loss_mask[:num_agents_considered])
        seq_list.append(curr_seq[:num_agents_considered])
        seq_list2.append(curr_seq2[:num_agents_considered])
        seq_list3.append(curr_seq3[:num_agents_considered])
        
        seq_list_rel.append(curr_seq_rel[:num_agents_considered])
        seq_list_rel2.append(curr_seq_rel2[:num_agents_considered])
        seq_list_rel3.append(curr_seq_rel3[:num_agents_considered])

num_seq = len(seq_list)
        
seq_list = np.concatenate(seq_list, axis=0)
seq_list2 = np.concatenate(seq_list2, axis=0)
seq_list3 = np.concatenate(seq_list3, axis=0)
seq_list_rel = np.concatenate(seq_list_rel, axis=0)
seq_list_rel2 = np.concatenate(seq_list_rel2, axis=0)
seq_list_rel3 = np.concatenate(seq_list_rel3, axis=0)

loss_mask_list = np.concatenate(loss_mask_list, axis=0)
non_linear_agent = np.asarray(non_linear_agent)

# Convert numpy -> Torch Tensor
obs_traj = torch.from_numpy(
    seq_list[:, :, :obs_len]).type(torch.float)
obs_state = torch.from_numpy(
    seq_list2[:, :, :obs_len]).type(torch.float)
# obs_traffic = torch.from_numpy(
#     seq_list3[:, :, :obs_len]).type(torch.float)

pred_traj = torch.from_numpy(
    seq_list[:, :, obs_len:]).type(torch.float)
pred_state = torch.from_numpy(
    seq_list2[:, :, obs_len:]).type(torch.float)
# pred_traffic = torch.from_numpy(
#     seq_list3[:, :, obs_len:]).type(torch.float)

obs_traj_rel = torch.from_numpy(
    seq_list_rel[:, :, :obs_len]).type(torch.float)
obs_state_rel = torch.from_numpy(
    seq_list_rel2[:, :, :obs_len]).type(torch.float)
# obs_traffic_rel = torch.from_numpy(
#     seq_list_rel3[:, :, :obs_len]).type(torch.float)

pred_traj_rel = torch.from_numpy(
    seq_list_rel[:, :, obs_len:]).type(torch.float)
pred_state_rel = torch.from_numpy(
    seq_list_rel2[:, :, obs_len:]).type(torch.float)
# pred_traffic_rel = torch.from_numpy(
#     seq_list_rel3[:, :, obs_len:]).type(torch.float)

loss_mask = torch.from_numpy(loss_mask_list).type(torch.float)
non_linear_agent = torch.from_numpy(non_linear_agent).type(torch.float)
cum_start_idx = [0] + np.cumsum(num_agents_in_seq).tolist()
seq_start_end = [
    (start, end)
    for start, end in zip(cum_start_idx, cum_start_idx[1:])
]

def len():              # len(train_dset)
    return num_seq

def getitem(index):
    start, end = seq_start_end[index]
    out = [
        obs_traj[start:end, :], 
        obs_state[start:end, :],
        pred_traj[start:end, :],
        pred_state[start:end, :],
        
        obs_traj_rel[start:end, :], 
        obs_state_rel[start:end, :], 
        pred_traj_rel[start:end, :],
        pred_state_rel[start:end, :],
        
        non_linear_agent[start:end], 
        loss_mask[start:end, :]
    ]
    return out

In [24]:
get1 = getitem(1)
# print(type(get1)) # list

# obs_traj
print((get1[0].shape)) # torch.Size([16, 2, 8])
print(get1[0])

torch.Size([16, 2, 8])
tensor([[[ 538803.6875,  538805.8125,  538807.7500,  538809.5625,  538811.1875,
           538812.6250,  538813.9375,  538815.2500],
         [4813986.0000, 4813987.0000, 4813987.5000, 4813988.5000, 4813989.5000,
          4813990.0000, 4813990.5000, 4813991.0000]],

        [[ 538853.0625,  538853.0625,  538853.0625,  538853.0625,  538853.0625,
           538853.0625,  538853.0625,  538853.0625],
         [4814029.5000, 4814029.5000, 4814029.5000, 4814029.5000, 4814029.5000,
          4814029.5000, 4814029.5000, 4814029.5000]],

        [[ 538809.6875,  538812.2500,  538814.3750,  538816.0625,  538817.4375,
           538818.3750,  538818.9375,  538819.0000],
         [4813996.0000, 4813997.5000, 4813998.5000, 4813999.0000, 4814000.0000,
          4814000.5000, 4814000.5000, 4814000.5000]],

        [[ 538851.0625,  538851.0625,  538851.0625,  538851.0625,  538851.0625,
           538851.0625,  538851.0625,  538851.0625],
         [4814024.5000, 4814024.5000, 48

In [26]:
# obs_state
print((get1[1].shape)) # torch.Size([16, 4, 8])
print(get1[1])

torch.Size([16, 4, 8])
tensor([[[ 2.0823e+01,  2.0184e+01,  1.8717e+01,  1.7034e+01,  1.5171e+01,
           1.3683e+01,  1.2878e+01,  1.2177e+01],
         [-1.0085e+00, -8.6460e-01, -1.1115e+00, -1.2640e+00, -1.2352e+00,
          -7.6780e-01, -4.2880e-01, -6.4190e-01],
         [ 6.2000e-03,  5.4000e-03,  8.9000e-03,  1.3100e-02,  2.4600e-02,
           4.2000e-02,  1.3300e-02, -8.9300e-02],
         [ 4.3470e-01,  4.3440e-01,  4.3390e-01,  4.3330e-01,  4.3170e-01,
           4.2820e-01,  4.2540e-01,  4.3190e-01]],

        [[ 2.8000e-03,  2.0000e-03,  1.9000e-03,  1.7000e-03,  2.3000e-03,
           1.9000e-03,  1.2000e-03,  1.2000e-03],
         [ 2.3000e-03,  8.0000e-04, -1.4000e-03,  1.0000e-04, -1.9000e-03,
          -8.0000e-04, -1.5000e-03,  5.0000e-04],
         [-4.9000e-03, -2.9000e-03, -1.4000e-03, -2.3000e-03,  2.7000e-03,
           5.1000e-03,  2.6000e-03, -9.0000e-04],
         [ 3.5949e+00,  3.5949e+00,  3.5949e+00,  3.5949e+00,  3.5949e+00,
           3.5949e+00,  3

In [27]:
# pred_traj
print((get1[2].shape)) # torch.Size([16, 2, 12])
print(get1[2])

torch.Size([16, 2, 12])
tensor([[[ 538816.4375,  538817.4375,  538818.3125,  538819.0625,  538819.6875,
           538820.1875,  538820.5625,  538820.8750,  538821.1875,  538821.4375,
           538821.6875,  538821.9375],
         [4813991.5000, 4813992.0000, 4813992.5000, 4813993.0000, 4813993.5000,
          4813993.5000, 4813994.0000, 4813994.0000, 4813994.0000, 4813994.5000,
          4813994.5000, 4813994.5000]],

        [[ 538853.0625,  538853.0625,  538853.0625,  538853.0625,  538853.0625,
           538853.0625,  538853.0625,  538853.0625,  538853.0625,  538853.0625,
           538853.0625,  538853.0625],
         [4814029.5000, 4814029.5000, 4814029.5000, 4814029.5000, 4814029.5000,
          4814029.5000, 4814029.5000, 4814029.5000, 4814029.5000, 4814029.5000,
          4814029.5000, 4814029.5000]],

        [[ 538819.0000,  538819.0000,  538819.0625,  538819.1250,  538819.1250,
           538819.1250,  538819.1250,  538819.1250,  538819.1250,  538819.1250,
           53881

In [28]:
# pred_state
print((get1[3].shape)) # torch.Size([16, 4, 12])
print(get1[3])

torch.Size([16, 4, 12])
tensor([[[ 1.1014e+01,  9.5965e+00,  8.2141e+00,  6.8797e+00,  5.6448e+00,
           4.4696e+00,  3.5585e+00,  3.0516e+00,  2.8543e+00,  2.5968e+00,
           2.2801e+00,  2.2973e+00],
         [-9.3730e-01, -9.9110e-01, -9.4260e-01, -8.9640e-01, -8.4630e-01,
          -7.5310e-01, -4.9050e-01, -2.0720e-01, -1.0730e-01, -2.5060e-01,
          -1.4550e-01,  1.6890e-01],
         [-1.9370e-01, -1.7970e-01, -5.4200e-02,  3.9800e-02,  5.7800e-02,
           4.3100e-02,  3.3100e-02,  2.6900e-02,  2.1200e-02,  1.9900e-02,
           1.8600e-02,  1.7200e-02],
         [ 4.5220e-01,  4.7350e-01,  4.8380e-01,  4.8240e-01,  4.7200e-01,
           4.5830e-01,  4.4530e-01,  4.3440e-01,  4.2500e-01,  4.1530e-01,
           4.0660e-01,  4.0130e-01]],

        [[ 1.9000e-03,  1.9000e-03,  2.0000e-03,  2.5000e-03,  3.2000e-03,
           3.4000e-03,  2.8000e-03,  2.0000e-03,  1.1000e-03,  1.3000e-03,
           1.1000e-03,  1.4000e-03],
         [-7.0000e-04, -1.2000e-03, -1.

In [29]:
# obs_traj_rel
print((get1[4].shape)) # torch.Size([16, 2, 8])
print(get1[4])

torch.Size([16, 2, 8])
tensor([[[ 0.0000,  2.1300,  1.9600,  1.8100,  1.6400,  1.4200,  1.3200,
           1.2900],
         [ 0.0000,  0.9900,  0.9100,  0.8400,  0.7500,  0.6600,  0.6000,
           0.5800]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000]],

        [[ 0.0000,  2.5500,  2.1000,  1.7300,  1.3400,  0.9600,  0.5200,
           0.1100],
         [ 0.0000,  1.2200,  1.0100,  0.8400,  0.6600,  0.4800,  0.2600,
           0.0500]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000]],

        [[ 0.0000,  0.4200,  0.3900, 

In [31]:
# obs_state_rel
print((get1[5].shape)) # torch.Size([16, 2, 8])
print(get1[5])

torch.Size([16, 4, 8])
tensor([[[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0.,

In [32]:
# pred_traj_rel
print((get1[6].shape)) # torch.Size([16, 2, 12])
print(get1[6])

torch.Size([16, 2, 12])
tensor([[[ 1.1900,  1.0200,  0.8700,  0.7400,  0.6100,  0.5000,  0.3900,
           0.3100,  0.3000,  0.2800,  0.2500,  0.2100],
         [ 0.5500,  0.5000,  0.4700,  0.4100,  0.3200,  0.2500,  0.2000,
           0.1400,  0.1400,  0.1300,  0.1100,  0.0800]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[-0.0400,  0.0300,  0.0800,  0.0500,  0.0000, -0.0100, -0.0100,
           0.0100,  0.0000,  0.0000,  0.0000,  0.0000],
         [-0.0200,  0.0000,  0.0400,  0.0200,  0.0000,  0.0000, -0.0100,
           0.0000,  0.0100,  0.0000, -0.0100,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.

In [33]:
# pred_state_rel
print((get1[7].shape)) # torch.Size([16, 4, 12])
print(get1[7])

torch.Size([16, 4, 12])
tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0.

In [34]:
# non_linear_agent
print((get1[8].shape)) # torch.Size([16])
print(get1[8])

torch.Size([16])
tensor([1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0.])


In [35]:
# loss_mask
print((get1[9].shape)) # torch.Size([16, 20])
print(get1[9])

torch.Size([16, 20])
tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       